## Construção de um Dashboar com Python e Plotly-Dash (Part 2)

Nesta parte iremos procurar construir um dashboard com elementos básicos de interação com o utilizador bem como analisar algumas opções/ferramentas para costumizar o desenho da interface.




## Directorias


In [1]:
#Colocar o path correto
path = "C:/.../"

path_projeto_ICD2021 = "TRABALHO_AULAS/AL20212022/ICD_202120221/"

path_dados = path + path_projeto_ICD2021 + "projeto_ICD2021/dados/" 

## Ativar as livrarias necessárias

In [2]:

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output


from jupyter_dash import JupyterDash

import plotly.express as px

import pandas as pd
import numpy



## PreProcessamento


In [3]:
scopus_api_search_db = pd.read_csv(path_dados+'scopus_api_search_db.csv')  
print(scopus_api_search_db.columns)
scopus_api_search_db[:3]

Index(['Unnamed: 0', 'dc:title', 'prism:doi', 'dc:creator', 'citedby-count',
       'openaccess', 'affilname', 'affiliation-city', 'affiliation-country'],
      dtype='object')


,Unnamed: 0,dc:title,prism:doi,dc:creator,citedby-count,openaccess,affilname,affiliation-city,affiliation-country
0,0,Rapid Earthquake Loss Estimation Model for Alg...,10.1080/15583058.2021.1958394,Boukri M.,0,0,Centre National de Recherche Appliquée en Géni...,Algiers,Algeria
1,1,An automatic tool for the determination of hou...,10.3390/su14010309,Tajani F.,0,1,Sapienza Università di Roma,Rome,Italy
2,2,"Interventions promoting uptake of water, sanit...",10.1002/cl2.1194,Chirgwin H.,0,1,International Initiative for Impact Evaluation,"Washington, D.C.",United States


### Processamento de dados adicionais

In [42]:
scopus_api_search_db_pivot1 = scopus_api_search_db.pivot_table(index =['affiliation-country'], 
                       values =['citedby-count'], 
                       aggfunc = {'sum', 'count'} )

scopus_api_search_db_pivot1.columns = [' '.join(col).strip() for col in scopus_api_search_db_pivot1.columns.values]
scopus_api_search_db_pivot1.reset_index(inplace=True)
scopus_api_search_db_pivot1['CitationAvrg'] = scopus_api_search_db_pivot1['citedby-count sum'] / scopus_api_search_db_pivot1['citedby-count count']
scopus_api_search_db_pivot1

,affiliation-country,citedby-count count,citedby-count sum,CitationAvrg
0,Algeria,8,64,8.000000
1,Australia,16,944,59.000000
2,Brazil,8,28,3.500000
3,China,15,147,9.800000
4,Germany,16,147,9.187500
5,Hong Kong,7,586,83.714286
6,Italy,8,219,27.375000
7,Japan,7,51,7.285714
8,Lithuania,8,52,6.500000
9,New Zealand,8,124,15.500000


In [54]:
# Prepare the first graph; it could also be done in the app.layout command but it is easier but hopefully
# the code will be easier to understand

fig1 = px.bar(scopus_api_search_db, x="openaccess", y = 'citedby-count', 
                title="Citations by type of publication"
                )
                
fig1.update_layout(showlegend=False)
fig1.update_traces(marker_line_width=0)
fig1.update_xaxes(type='category')

In [59]:
fig1a = px.histogram(scopus_api_search_db, x="openaccess", y = 'citedby-count', 
             #barmode='group',
             histfunc='sum', #histfunc='avg',
             height=400)

fig1a.update_xaxes(type='category')
fig1a.show()

In [44]:
fig2= px.pie(scopus_api_search_db_pivot1, values='citedby-count count', names='affiliation-country')
fig2.show()

In [61]:
fig2b= px.pie(scopus_api_search_db, values='citedby-count', names='affilname')
fig2b.show()

## Intereção I

### Criar o layout base e os elementos (gráficos e outros) a utilizar

In [71]:
app = JupyterDash(__name__)
app.layout = html.Div(
    children=[
        html.H1(children="Scopus dataset analysis",),
        html.P(
            children="... I will put a description here later ..."
        ),
        html.Hr(),
        html.H4(children="Analyse by country:", ),
        html.Div(
            [
                html.Div(children="country"),
                
                dcc.Dropdown(
                    id="country-filter",
                    style={"width": "40%"},
                    options=[
                        {"label": affcountry, "value": affcountry}
                        for affcountry in scopus_api_search_db["affiliation-country"].unique()
                    ],
                    value="Affiliation Country",
                    clearable = False,
                ),

            ],
        ),

       html.Hr(),

        #most of the code will go in from here
        html.Div( [
            dcc.Graph(id="CountryPlot",  style={'width': '45%', 'display': 'inline-block'}),
            ]
        ),
    ]
)

### Interação básica com o utilizador

In [72]:
# callback function
@app.callback(
    [
        Output("CountryPlot", "figure"),
    ],
    [
        Input("country-filter", "value"),
    ],
)

def update_figure(affcountry):

    filteredData = scopus_api_search_db[scopus_api_search_db["affiliation-country"] == affcountry]
    figure1 = px.bar(filteredData, x="openaccess", y = 'citedby-count', 
                title="Citations by type of acess") #,color='affiliation-country'
    figure1.update_layout(transition_duration=500)

    # return [figure1, figure2, figure3]
    return [figure1]

## Interação II 
»» um pouco mais sofisticado

In [79]:
app = JupyterDash(__name__)
app.layout = html.Div(
    children=[
        html.H1(children="Scopus dataset analysis",),
        html.P(
            children="... I will put a description here later ..."
        ),
        html.Hr(),
        html.H4(children="Analyse by country:", ),
        html.Div(
            [
                html.Div(children="country"),
                
                dcc.Dropdown(
                    id="country-filter",
                    style={"width": "40%"},
                    options=[
                        {"label": affcountry, "value": affcountry}
                        for affcountry in scopus_api_search_db["affiliation-country"].unique()
                    ],
                    value="affcountry1",
                    clearable = False,
                ),

            ],
        ),

       html.Hr(),

        #most of the code will go in from here
        html.Div( [
            dcc.Graph(id="CountryPlot",  style={'width': '45%', 'display': 'inline-block'}),
            ]
        ),

        html.Div( [
            dcc.Graph(id="AffiliationPlot",  style={'width': '45%', 'display': 'inline-block'}),
            ]
        ),
    ]
)

In [80]:
# callback function
@app.callback(
    [
        Output("CountryPlot", "figure"),
        Output("AffiliationPlot", "figure"),
    ],
    [
        Input("country-filter", "value"),
    ],
)

def update_figure(affcountry1):

    filteredData = scopus_api_search_db[scopus_api_search_db["affiliation-country"] == affcountry1]
    figure1 = px.bar(filteredData, x="openaccess", y = 'citedby-count', 
                title="Citations by type of acess") #,color='affiliation-country'
    figure1.update_layout(transition_duration=500)

    figure2= px.pie(filteredData, values='citedby-count', names='affilname')
    figure2.update_layout(transition_duration=500)

    # return [figure1, figure2, figure3]
    return [figure1, figure2]

## Extra: Layout "bootstrap"
https://dash-bootstrap-components.opensource.faculty.ai/

In [85]:
import dash_bootstrap_components as dbc

In [ ]:
#app = JupyterDash(__name__)

In [135]:
#app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])
#app = JupyterDash(__name__)
#external_stylesheets=[dbc.themes.SLATE]
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])

controls = dbc.Card(
    [
       html.Hr(),
        html.H4(children="Analyse by country:", ),
        html.Div(
            [
                html.Div(children="country"),
                
                dcc.Dropdown(
                    id="country-filter",
                    style={"width": "40%"},
                    options=[
                        {"label": affcountry, "value": affcountry}
                        for affcountry in scopus_api_search_db["affiliation-country"].unique()
                    ],
                    value="affcountry1",
                    clearable = False,
                ),

            ],
        ),
    ],
    body=True,
)

app.layout = dbc.Container(
    html.Div(
                
            [
                dbc.Row(
                    html.H1("Scopus dataset analysis II"),
                    
                     ),
                dbc.Row(
                    dbc.Col(controls, width=12),
                ),
                dbc.Row(
                    [
                        dbc.Col(dcc.Graph(id="CountryPlot"), width=6),
                        dbc.Col(dcc.Graph(id="AffiliationPlot"), width=6),
                    ]
                    #align="center",
                ),
            ]
        )
    
    
)




@app.callback(
    [
        Output("CountryPlot", "figure"),
        Output("AffiliationPlot", "figure"),
    ],
    [
        Input("country-filter", "value"),
    ],
    
    
)

def update_figure(affcountry1):

    filteredData = scopus_api_search_db[scopus_api_search_db["affiliation-country"] == affcountry1]
    figure1 = px.bar(filteredData, x="openaccess", y = 'citedby-count', 
                title="Citations by type of acess") #,color='affiliation-country'
    figure1.update_xaxes(type='category')
    figure1.update_layout(transition_duration=500)

    figure2= px.pie(filteredData, values='citedby-count', names='affilname')
    figure2.update_layout(transition_duration=500)

    # return [figure1, figure2, figure3]
    return [figure1, figure2]


# Correr o dashboard num servidor local

In [136]:
# run the server
app.run_server(mode='external', debug=True, port=8061)
#app.run_server(debug=True, port=8888)

Dash app running on http://127.0.0.1:8061/


# Desafios


**Ex01:** Criar componentes de interesse para visualização dos dados recolhidos
1. Preparar os dados, se e quando necessário;
1. Definir um layout e addicionar as componentes (gráficos);
1. Implementar um esquema de interação com os utilizadores.
Repetir para cada componente


**Ex02:** Usar o Dashboard para responder a algumas questões de investigação
1. Can you find a rule to classify 'Iris-setosa' flowers with 100% accuracy?
1. What rule would you use to distinguish "Iris-versicolor" from 'Iris-virginica' flowers?

**Ex03:** Desafio adicional: introduzir funcionalidades de interação adicionais (por exemplo, acrescentar critérios de filtro adicionais )